畫圖套件

In [1]:
#下載資料套件
import urllib3
import requests
from bs4 import BeautifulSoup
#處理資料套件
import pandas as pd
from datetime import datetime
from io import StringIO
import json
#仔仔
from headers import *
import controllers

import importlib
importlib.reload(controllers)
controller = controllers.FuturesDataController()

In [2]:
update_date = "2024/10/30"
controller.update_twse_data(update_date)
controller.update_taifex_data(update_date)

爬取日期: 20241030
成功連接到 MongoDB!
成功插入 1 條文檔
MongoDB 連接已關閉
爬取日期: 2024/10/30
爬取成功
轉換成功


/Users/hayden/Documents/pyDB/controllers.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter[col] = pd.to_numeric(df_filter[col].astype(str).str.replace(',', ''), errors='coerce')
/Users/hayden/Documents/pyDB/controllers.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter[col] = pd.to_numeric(df_filter[col].astype(str).str.replace(',', ''), errors='coerce')
/Users/hayden/Documents/pyDB/controllers.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

成功連接到 MongoDB!
成功插入 6721d80886eb7057a86616ae 條文檔
MongoDB 連接已關閉


'成功更新 9 條記錄到 MongoDB'

In [3]:
url = "https://www.taifex.com.tw/cht/3/futContractsDateAh"
date="2024/09/09"
params = {
             'queryType': '2',
             'goDay': '',
             'doQuery': '1',
             'dateaddcnt': '',
             'queryDate': date
         }
response = requests.post(url,headers=TAIFEX_OI,data=params)
response.raise_for_status()


In [ ]:
soup = BeautifulSoup(response.content,'html.parser')
table = soup.table
df = pd.read_html(StringIO(str(table)))
df = df[0]
df

In [ ]:
#df.drop(columns = df.columns[0],axis=1,inplace=True)
df.columns=['序號','商品名稱','身份別','多方交易口數','多方交易金額','空方交易口數','空方交易金額','交易淨口數','交易淨金額']
df = df[[col for col in df.columns if col!='序號']]

df_filter = df.loc[df['商品名稱'].isin(['臺股期貨','小型臺指期貨','微型台指期貨'])]

numeric_columns = ['多方交易口數','多方交易金額','空方交易口數','空方交易金額','交易淨口數','交易淨金額']

for col in numeric_columns:
    df_filter[col] = pd.to_numeric(df_filter[col].astype(str).str.replace(',',''),errors='coerce')

df_filter

In [ ]:
df = controller.crawler_taifex_vix("2024/09/09")
df

In [ ]:
date = "2024/09/09"
dates = date.split('/')
params = dates[0]+dates[1].zfill(2)
url = "https://www.taifex.com.tw/file/taifex/Dailydownload/vix/log2data/{params}new.txt"
url

主程式碼

In [ ]:
url = "https://www.taifex.com.tw/file/taifex/Dailydownload/vix/log2data/202409new.txt"
response = requests.post(url)
response.encoding = "big5"
df = pd.read_csv(StringIO(response.text),sep=r"\t+",index_col = False,encoding='big5',skiprows=[1],engine='python')
df = df.rename(columns={'交易日期':'date'})
df['date'] = pd.to_datetime(df['date'],format="%Y%m%d")
df = df[['date','臺指選擇權波動率指數']]

In [ ]:
#df = df[['date','臺指選擇權波動率指數']]
df

In [ ]:
url = "https://www.taifex.com.tw/cht/3/futContractsDate"
#querydate = datetime.now().strftime("%Y/%m/%d")
querydate = "2024/08/26"
print(f"爬取日期: {querydate}")
params = {
        'queryType': '2',
        'goDay': '',
        'doQuery': '1',
        'dateaddcnt': '',
        'queryDate': querydate
    }

response = requests.post(url,headers=TAIFEX_OI,data=params)

if response.status_code == 200:
    dfs = pd.read_html(StringIO(response.text))
    if len(dfs) > 0:
        print(len(dfs))
        tdf = dfs[0]
        print("show")
        tdf
    
    else:
        print("沒有找到數據表格")
        
else:
    print(f"請求失敗，狀態碼：{response.status_code}")
        


In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.table
table


In [ ]:
df = pd.read_html(StringIO(str(table)))
df

In [ ]:
print(len(df))
df = df[0]

In [217]:
#df.to_csv('test.csv', index=False, encoding='utf-8-sig')

In [ ]:
df.drop(columns = df.columns[0],axis=1,inplace=True)
df.head()


In [ ]:
df.columns=['商品名稱','身份別','多方交易口數','多方交易金額','空方交易口數','空方交易金額','交易淨口數','交易淨金額','多方未平倉口數','多方未平倉金額','空方未平倉口數','空方未平倉金額','未平倉淨口數','未平倉淨金額']
df

#篩選指定的期貨類型

In [ ]:
df_filter = df.loc[df['商品名稱'].isin(['臺股期貨','小型臺指期貨','微型臺指期貨'])]
df_filter

#載入MongoDB套件

In [221]:
from pymongo import MongoClient
from pymongo.server_api import ServerApi
import json

In [ ]:


# 從配置文件讀取連接信息
with open('config.json') as config_file:
    config = json.load(config_file)

mongo_user = config['mongo_user']
mongo_pass = config['mongo_pass']
mongo_host = config['mongo_host']

# 構建連接 URI
db_uri = f"mongodb+srv://{mongo_user}:{mongo_pass}@{mongo_host}/?retryWrites=true&w=majority&appName=FDB"

# 創建客戶端連接
client = MongoClient(db_uri, server_api=ServerApi('1'))

try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)



In [ ]:
# 進行數據操作...
client = MongoClient(db_uri, server_api=ServerApi('1'))
db = client['findata']
collection = db['txf_oi']

#檢查是否已存在該日期數據
existing_data = collection.find_one({'date':querydate})
if existing_data:
    print(f"{querydate}的數據已存在,跳過存入")


data_dict = df_filter.to_dict('records')

for record in data_dict:
    record['date'] = datetime.strptime(querydate,"%Y/%m/%d")



result = collection.insert_many(data_dict)
print(f"成功插入 {len(result.inserted_ids)} 條文檔")

client.close()



In [ ]:

# 可選：驗證插入的數據
# 重新連接並查詢數據
client = MongoClient(db_uri, server_api=ServerApi('1'))
db = client['findata']
collection = db['txf_oi']

# 查詢並打印前幾條數據
for doc in collection.find().limit(5):
    print(doc)


client.close()